In [9]:


import joblib
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('../data/cyberbullying_preprocessed.csv')
data = [tuple(x) for x in df.values]
print('Número de datos cargados: {num}'.format(num=len(data)))

Número de datos cargados: 80984


In [3]:
# Divido los datos en dos listas 
#     X: los mensajes de texto
#     y: las etiquetas

X = [doc[2] for doc in data]
y = [doc[1] for doc in data]

print(pd.DataFrame({'X': X, 'y': y}))

                                                       X  y
0                 word katandandre food crapilicious mkr  0
1      aussietv white mkr theblock imacelebrityau tod...  0
2                        classy whore red velvet cupcake  0
3            meh thank head concerned angry dude twitter  0
4      isis account pretend kurdish account like isla...  0
...                                                  ... ..
80979                                     happy birthday  0
80980  agree awful make sense regardless mistake dese...  0
80981  call yesterday guidance counselor office think...  0
80982                                              thank  0
80983                 think find good group help refocus  0

[80984 rows x 2 columns]


In [4]:
# Cargamos el vector TF-IDF y ajustamos los datos
tfidf_vectorizer = joblib.load('../models/tfidf_vectorizer.pkl')
X = tfidf_vectorizer.transform(X)

# Dividimos los datos en dos conjuntos: entrenamiento y test
- El conjunto de entrenamiento se utiliza para ajustar el modelo
- El conjunto de test se utiliza para evaluar el modelo
- La proporción de los datos que se utilizan para el test es del 20%
- La semilla aleatoria se fija en 0 para que los resultados sean reproducibles
- Se imprime el número de mensajes que se utilizarán para el entrenamiento y el test

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print('Número de Mensajes para el entrenamiento: {num}'.format(num=X_train.shape[0]))
print('Número de Mensajes para el test: {num}'.format(num=X_test.shape[0]))

Número de Mensajes para el entrenamiento: 64787
Número de Mensajes para el test: 16197


# Clasificadores
Hacemos uso de los siguientes clasificadores:
- Naive Bayes.
    - Multinomial Naive Bayes
    - Bernoulli Naive Bayes
- Regresión Logística.
- Máquinas de Soporte Vectorial (SVM).
    - Lineal
    - Polinómico
    - Radial
    - Sigmoide
- Random Forest.
    - Random Forest con profundidad de 20
    - Random Forest con profundidad de 50

### Naive Bayes
Es un clasificador probabilístico que se basa en el teorema de Bayes. Se utilizan diferentes tipos de Naive Bayes:
- Multinomial Naive Bayes: Se utiliza para datos discretos.
- Bernoulli Naive Bayes: Se utiliza para datos binarios.

### Regresión Logística
Es un clasificador lineal que se utiliza para problemas de clasificación binaria y multiclase.

### Máquinas de Soporte Vectorial (SVM)
Es un clasificador lineal que se utiliza para problemas de clasificación binaria y multiclase.
- Se utilizan diferentes kernels: lineal, polinómico, radial y sigmoide.

### Random Forest
Es un clasificador que se basa en la combinación de árboles de decisión. Se utilizan diferentes profundidades para los árboles de decisión.

In [6]:
# Importamos los clasificadores
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Inicializamos los clasificadores
multinomial_nb_classifier = MultinomialNB()
bernoulli_nb_classifier = BernoulliNB()

logistic_regression_classifier = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=1000)

linear_svm_classifier = SVC(kernel='linear')
poly_svm_classifier = SVC(kernel='poly')
rbf_svm_classifier = SVC(kernel='rbf')
sigmoid_svm_classifier = SVC(kernel='sigmoid')

random_forest_depth_20_classifier = RandomForestClassifier(n_estimators=500, bootstrap=True, criterion='gini',
                                                           max_depth=20, random_state=0)
random_forest_depth_50_classifier = RandomForestClassifier(n_estimators=500, bootstrap=True, criterion='gini',
                                                           max_depth=50, random_state=0)

In [14]:
# Creamos un diccionario con los clasificadores
classifiers = {'Multinomial NB': multinomial_nb_classifier,
               'Bernoulli NB': bernoulli_nb_classifier,
               'Logistic Regression': logistic_regression_classifier,
               'Linear SVM': linear_svm_classifier,
               'Polynomial SVM': poly_svm_classifier,
               'RBF Kernel SVM': rbf_svm_classifier,
               'Sigmoid Kernel SVM': sigmoid_svm_classifier,
               'Random Forest Depth 20': random_forest_depth_20_classifier,
               'Random Forest Depth 50': random_forest_depth_50_classifier}

In [8]:
# Entrenamos los clasificadores
print(f"Entramiento de los clasificadores\n{'=' * 30}")
for classifier_name, classifier in classifiers.items():
    print(f"Entrenando {classifier_name}...")
    classifier.fit(X_train, y_train)

Entramiento de los clasificadores
Entrenando Multinomial NB...
Entrenando Bernoulli NB...
Entrenando Logistic Regression...
Entrenando Linear SVM...
Entrenando Polynomial SVM...
Entrenando RBF Kernel SVM...
Entrenando Sigmoid Kernel SVM...
Entrenando Random Forest Depth 20...
Entrenando Random Forest Depth 50...


In [12]:
# Guardamos los clasificadores entrenados
for classifier_name, classifier in classifiers.items():
    print(f"Guardando {classifier_name}...")
    joblib.dump(classifier, f'../models/{classifier_name.lower().replace(" ", "_")}.pkl')

Guardando Multinomial NB...
Guardando Bernoulli NB...
Guardando Logistic Regression...
Guardando Linear SVM...
Guardando Polynomial SVM...
Guardando RBF Kernel SVM...
Guardando Sigmoid Kernel SVM...
Guardando Random Forest Depth 20...
Guardando Random Forest Depth 50...


In [15]:
# Cargamos los clasificadores entrenados
for classifier_name in classifiers.keys():
    print(f"Cargando {classifier_name}...")
    classifiers[classifier_name] = joblib.load(f'../models/{classifier_name.lower().replace(" ", "_")}.pkl')

Cargando Multinomial NB...
Cargando Bernoulli NB...
Cargando Logistic Regression...
Cargando Linear SVM...
Cargando Polynomial SVM...
Cargando RBF Kernel SVM...
Cargando Sigmoid Kernel SVM...
Cargando Random Forest Depth 20...
Cargando Random Forest Depth 50...


In [17]:
# Evaluamos los atributos (accuracy, precision, recall, f1) de los clasificadores, tanto los datos de entrenamiento y prueba.
# accuracy: mide la fracción de predicciones correctas
# precision: mide la fracción de predicciones correctas de la clase positiva
# recall: mide la fracción de instancias positivas que fueron correctamente clasificadas
# f1: media armónica de precision y recall
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


def evaluation_metrics(model, name, X_train, y_train, X_test, y_test):
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)

    train_precision = precision_score(y_train, y_train_pred, average='weighted')
    test_precision = precision_score(y_test, y_test_pred, average='weighted')

    train_recall = recall_score(y_train, y_train_pred, average='weighted')
    test_recall = recall_score(y_test, y_test_pred, average='weighted')

    train_f1 = f1_score(y_train, y_train_pred, average='weighted')
    test_f1 = f1_score(y_test, y_test_pred, average='weighted')

    return [name, train_accuracy, test_accuracy, train_precision, test_precision, train_recall, test_recall, train_f1,
            test_f1]

In [18]:
# Calculamos as metricas de evaluacion para cada clasificador
evaluation_results = []
for classifier_name, classifier in classifiers.items():
    # print(f"Calculando métricas de evaluación para {classifier_name}...")
    evaluation_results.append(evaluation_metrics(classifier, classifier_name, X_train, y_train, X_test, y_test))

df_evaluation_results = pd.DataFrame.from_dict(evaluation_results)
df_evaluation_results.columns = ['Model', 'Train Accuracy', 'Test Accuracy', 'Train Precision', 'Test Precision',
                                  'Train Recall', 'Test Recall', 'Train F1', 'Test F1']
df_evaluation_results

Calculando métricas de evaluación para Multinomial NB...
Calculando métricas de evaluación para Bernoulli NB...
Calculando métricas de evaluación para Logistic Regression...
Calculando métricas de evaluación para Linear SVM...
Calculando métricas de evaluación para Polynomial SVM...
Calculando métricas de evaluación para RBF Kernel SVM...
Calculando métricas de evaluación para Sigmoid Kernel SVM...
Calculando métricas de evaluación para Random Forest Depth 20...
Calculando métricas de evaluación para Random Forest Depth 50...


,Model,Train Accuracy,Test Accuracy,Train Precision,Test Precision,Train Recall,Test Recall,Train F1,Test F1
0,Multinomial NB,0.745829,0.743903,0.742345,0.740215,0.745829,0.743903,0.742142,0.739885
1,Bernoulli NB,0.772068,0.770266,0.801503,0.799174,0.772068,0.770266,0.774798,0.773101
2,Logistic Regression,0.787164,0.777737,0.796736,0.787945,0.787164,0.777737,0.789233,0.779977
3,Linear SVM,0.787118,0.781256,0.811575,0.807505,0.787118,0.781256,0.789721,0.783986
4,Polynomial SVM,0.900119,0.781750,0.899891,0.781923,0.900119,0.781750,0.899966,0.781834
5,RBF Kernel SVM,0.875052,0.790579,0.881195,0.801505,0.875052,0.790579,0.876111,0.792764
6,Sigmoid Kernel SVM,0.726674,0.747855,0.735801,0.757207,0.726674,0.747855,0.729173,0.750251
7,Random Forest Depth 20,0.783166,0.768846,0.801845,0.785983,0.783166,0.768846,0.785766,0.771619
8,Random Forest Depth 50,0.822711,0.784713,0.862328,0.818175,0.822711,0.784713,0.824619,0.787278
